In [1]:
%matplotlib inline
import os
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import re
import scipy as sp
import json
import requests
import warnings
sns.set_context('notebook')
warnings.filterwarnings('ignore')

To make a hit movie,various features are deserved for detailed analysis, both value and non-value features. Also, for better visualization in this part, we use plotly library to get interactive figures of the relationships, which is also more suitable for later data story. The plotly figure may not been shown in the jupyter notebook, but they can been attatched in the folder named 'plotly_image'.

In [3]:
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
import plotly.figure_factory as ff
import plotly.tools as tls
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly
plotly.tools.set_credentials_file(username='zx_haowan', api_key='J4Agcce1dOjvqiKRTpoO')

In [4]:
merge_an=merge_raw.copy()
tmdb_data=pd.read_pickle('tmdb_data.pkl')
merge_an=merge_an.groupby('title').mean()
merge_an['title']=merge_an.index
merge_an=merge_an[['title','overall']]
merge_an=pd.merge(merge_an,tmdb_data,on='title')
merge_an.head(1)

,title,overall,cast,budget,genres,keywords,original_language,popularity,production_companies,production_countries,revenue,runtime,spoken_languages,vote_average,vote_count,director,release_year,profit,title_change
0,11:14,4.185714,"[Henry Thomas, Blake Heron, Barbara Hershey, H...",6000000,"[Crime, Drama, Thriller]","[alcohol, sex, robbery, secret, gun, ambulance...",en,15.048067,"[Firm Films, Media 8 Entertainment, MDP Worldw...","[Canada, United States of America]",0,86.0,[English],6.8,206,Greg Marcks,2003.0,-1.0,1


In [6]:
def get_count(col):
    temp_count = dict()
    for row in col:
        for temp_feature in row:
            if temp_feature in temp_count:
                temp_count[temp_feature] += 1
            else:
                temp_count[temp_feature] = 1
    return temp_count

In [7]:
def get_direcotr_count(col):
    temp_count = dict()
    for temp_director in col:
            if temp_director in temp_count:
                temp_count[temp_director] += 1
            else:
                temp_count[temp_director] = 1
    return temp_count

To make a hit movie, in addtion to simple numbers like budget and popularity, other text information is also informative. In this part we focus on the relationship between Amazon rating and non-value features like movie genres, directors and production companies.

The genres of movies are very informative, for example, in typical years , specific genres of movie tend to lead a fashion, and greatly affect the review, rating and other forms of evluation towards a film. First, an interactive evolution of movie genres is plotted.

In [194]:
genre_list=list(genres_stat.columns)
potantial_color=['#fbb4ae','#b3cde3','#ccebc5','#decbe4','#fed9a6','#c7eae5','#e5d8bd','#fddaec','#f2f2f2','#fbb4ae','#b3cde3','#ccebc5','#decbe4','#fed9a6','#c7eae5','#e5d8bd','#fddaec','#f2f2f2','#fbb4ae','#b3cde3','#ccebc5','#decbe4','#fed9a6','#c7eae5','#e5d8bd','#fddaec','#f2f2f2','#fbb4ae','#b3cde3','#ccebc5']
temp_line=[]
for ind, genre in enumerate(genre_list):
    temp_line.append(
        go.Scatter(
            x=genres_stat.index,
            y=genres_stat[genre],
            name=genre,legendgroup=genre,
            line = dict(color = (potantial_color[ind]),width = 3),), )
layout = go.Layout( autosize=True, title='Evoluation of Genres through Years',)
fig = go.Figure(data=temp_line,layout=layout)
plot(fig, show_link=False,filename='Evoluation of Genres through Years')
py.iplot(fig,filename='Evoluation of Genres through Years')

The plotly figure is stored in 'plotly_image/Evoluation of Genres through Years.html' in github.

Drama has always been a leading trend in history, followed by comedy and little attention is paid to foreign movies. The trend of other genres have been fluctuating through time and market of movies for that are usually changing.

Intuitvely, a genre will affect the rating and review length of a movie since a hot type will attract more people and arouse their thoughts for more review texts, so average rating and review length of each genre will be analyzed.

In [133]:
merge_an['review_length']=[len(temp_review) for temp_review in merge_an.reviewText]
genres_count = pd.Series(get_count(merge_an['genres']))

We dedine a function to extract the average value of a selected feature for each genre. The function can also be use in later analysis of production countries since they share the same sturcture in the original dataframe.

In [121]:
def get_avg_all(genres_count,genres,rev,merge_an):
    rev_array=[]
    for temp_genre in genres_count.index:
        count=rev_sum=0
        for i in range(len(merge_an)):
            if temp_genre in merge_an[genres].values[i]:
                count=count+1
                temp_rev=merge_an[rev].values[i]
                rev_sum=rev_sum+temp_rev
        if count==0: temp_avg_rev=0
        else: temp_avg_rev=rev_sum/count
        rev_array=rev_array+[temp_avg_rev]
    temp_pd=pd.DataFrame({genres:np.array(genres_count.index),rev:np.array(rev_array)})
    return temp_pd

We use the function to extract the feature values shown below.

In [141]:
merge_an=merge_an.fillna(0)
genre_overall=get_avg_all(genres_count,'genres','overall',merge_an)
genre_pop=get_avg_all(genres_count,'genres','popularity',merge_an)
temp_genre_pd=pd.merge(genre_overall,genre_pop,on='genres')
genre_len=get_avg_all(genres_count,'genres','review_length',merge_an)
temp_genre_pd=pd.merge(temp_genre_pd,genre_len,on='genres')
genre_rev=get_avg_all(genres_count,'genres','revenue',merge_an)
temp_genre_pd=pd.merge(temp_genre_pd,genre_rev,on='genres')
genre_va=get_avg_all(genres_count,'genres','vote_average',merge_an)
genre_pd=pd.merge(temp_genre_pd,genre_va,on='genres')
genre_pd.head(1)

,genres,overall,popularity,review_length,revenue,vote_average
0,Action,3.914473,29.624649,1012.696703,1.078980e+08,6.29199


We use a bar chart and line plot to show the distribution of average rating and review length.

In [231]:
#genre_pd=genre_pd.drop(9,axis=0)
va_fig=go.Bar(
    x=list(genre_pd.genres),
    y=genre_pd.overall.values,
    name='Average Rating',
    marker=go.Marker(color='#b3cde3')
)
len_fig=go.Scatter(
    x=list(genre_pd.genres),
    y=genre_pd.review_length.values,
    name='Average Length of Review', yaxis='y2',line = dict(color = ('#fbb4ae'),width = 3))
data=[va_fig,len_fig]
layout=go.Layout(
    title='Average Rating and Average Length of Review in All Genres',
    titlefont=dict(size=20),
    xaxis=dict(title='Genres' ),
    yaxis=dict(title='Average Rating',showgrid=False, ),
    yaxis2=dict(title='Average Length of Review',overlaying='y',showgrid=False,side='right'),
    legend=dict(orientation="h", x=.7,y=1.1,),)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig,filename='genres_rating_vs_review_length')

D:\python\lib\site-packages\plotly\graph_objs\_deprecations.py:426: DeprecationWarning:

plotly.graph_objs.Marker is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Marker
  - plotly.graph_objs.histogram.selected.Marker
  - etc.




The plotly figure is stored in 'plotly_image/genres_rating_vs_review_length.html' in github.

Most genres share the close average rating with Science Fiction rated highest. However, 
Visually there is no direct relationship between average rating and review length among different genres, at leat no linear relationship.

In [152]:
max_pop=max(genre_pd.popularity)
max_rev=max(genre_pd.revenue)
max_va=max(genre_pd.vote_average)
genre_pd['popularity']=genre_pd['popularity']/max_pop
genre_pd['revenue']=genre_pd['revenue']/max_rev
genre_pd['vote_average']=genre_pd['vote_average']/max_va

Also, bar charts of popularity, revenue and average vote are shown. To make the features more visible, we divide each feature by the maximum value in them, thus sharing a same range of (0,1).

In [229]:
potantial_color=['#b3cde3','#fbb4ae','#fed9a6']
data = []
for ind, value in enumerate(['popularity','revenue','vote_average']):
    data.append(
        go.Bar(
            x=list(genre_pd.genres),
            y=genre_pd[value],
            name=value, 
            marker = dict(color = (potantial_color[ind]),
            line = dict(color = (potantial_color[ind]),width = 3), ) )  )
layout = go.Layout(autosize=True,title='Popularity, Revenue and Average for All genres',)
fig = go.Figure(data=data,layout=layout)
py.iplot(fig,filename='Popularity, Revenue and Average for All genres')

The plotly figure is stored in 'plotly_image/genres_popularity_revenue_average.html' in github.

Director is also an important factor that lead to profit and review. Personally, if James Cameron directs a new movie, I will definitely go to the cinema for it and make my reviews after the payment. It's called the influence of celebrity. Compared with other crew or cast in movie procution, a director is perhaps the most important role in a movie, and we will analyze that.

In [202]:
director_count = pd.Series(get_direcotr_count(merge_an['director'])).sort_values(ascending=False)
director_count=director_count[1:]
director_an_df=pd.DataFrame(director_count)
director_an_df['director']=director_an_df.index
for temp_genres in genres_count.index:
    director_an_df[temp_genres]=0
for temp_overall in [1.0,2.0,3.0,4.0,5.0]:
    director_an_df['overall '+str(temp_overall)]=0

In [210]:
director_an_df[director_an_df.director=='Martin Scorsese']['overall 1.0'].values[0]=1

In [220]:
for temp_director in director_count.index:
    for i in range(len(merge_an)):
        if temp_director==merge_an.director.values[i]:
            temp_overall=merge_an.overall.values[i]
            director_an_df['overall '+str(temp_overall)][temp_director]=director_an_df['overall '+str(temp_overall)][temp_director]+1
            for temp_genre in merge_an.genres[i]:
                director_an_df[temp_genre][temp_director]=director_an_df[temp_genre][temp_director]+1
director_an_df.head(1)

,0,director,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,...,Science Fiction,TV Movie,Thriller,War,Western,overall 1.0,overall 2.0,overall 3.0,overall 4.0,overall 5.0
Martin Scorsese,1202,Martin Scorsese,1,0,0,0,823,0,1202,0,...,0,0,0,0,0,52,53,90,228,779


By getting the average rating for each director, we get the plot of top 10 directors that participate in most movies and their corresponding average rating.

In [185]:
director_count = pd.Series(get_direcotr_count(merge_an['director'])).sort_values(ascending=False)
director_count=director_count[1:]
director_count_df=pd.DataFrame(director_count)
director_count_df['director']=director_count_df.index
director_count_df['count']=director_count_df[0]
director_count_df.reset_index(inplace=True)
director_count_df=director_count_df[['director','count']]
director_count_df['total_rating']=0
for temp_director in merge_an.director:
    total_merge_temp_director=merge_an[merge_an['director']==temp_director]
    temp_total_rating=total_merge_temp_director.overall.sum()
    temp_index=director_count_df[director_count_df['director']==temp_director].index
    director_count_df.total_rating[temp_index]=total_merge_temp_director.overall.sum()
director_count_df['average_rating']=director_count_df['total_rating']/director_count_df['count']

In [227]:
director_count_df=director_count_df.iloc[:20]
layout = go.Layout(xaxis = dict(tickangle = 30,tickfont = dict(size = 10)),
                  yaxis=dict(type='linear',range=[1, 5],dtick=0.1 ),title='Average Rating for Top 20 Popular Directors')
data = [go.Bar(x=director_count_df.director, y=director_count_df.average_rating,
            marker=dict(
                color=['#fbb4ae','#b3cde3','#ccebc5','#decbe4','#fed9a6','#c7eae5','#e5d8bd','#fddaec','#f2f2f2','#fbb4ae','#b3cde3','#ccebc5','#decbe4','#fed9a6','#c7eae5','#e5d8bd','#fddaec','#f2f2f2','#fbb4ae','#b3cde3','#ccebc5','#decbe4','#fed9a6','#c7eae5','#e5d8bd','#fddaec','#f2f2f2','#fbb4ae','#b3cde3','#ccebc5']
            ),
            opacity=0.8
        )]
fig = go.Figure(data=data, layout=layout)
#plot(fig, show_link=True,filename='director_rating')
py.iplot(fig,filename='director_rating')

The plotly figure is stored in 'plotly_image/director_rating.html' in github.

Since these 10 directors participate in more movies than others, they have their own charms. All of them get a rating over 3.0 and most of them have a rating over 4.0, which can show their ability of directing movies to some degree, making them popular when making a movie.

In [78]:
director_va_profit=pd.merge(director_count_df,merge_an,on='director')
director_va_profit=director_va_profit[['director','average_rating','profit','vote_average']]
director_va_profit.fillna(0)
director_va_profit.head(1)

,director,average_rating,profit,vote_average
0,Martin Scorsese,4.207628,-1.0,6.6


The profit proportion and average vote in TMDB are also calculated for top 10 directors and then plotted.

In [86]:
director_va_profit=director_va_profit.groupby('average_rating').mean()
director_va_profit=director_va_profit.iloc[:10]
director_va_profit=pd.merge(director_count_df,director_va_profit,on='average_rating')
director_va_profit=director_va_profit[['director','average_rating','profit','vote_average']]
director_va_profit.head(2)

,director,average_rating,profit,vote_average
0,Brian De Palma,3.476032,0.837493,6.116667
1,Woody Allen,3.752109,4.734506,6.666667


In [228]:
potantial_color=['#b3cde3','#fbb4ae']
data = []
for ind, value in enumerate(['profit','vote_average']):
    data.append(
        go.Bar(
            x=list(director_va_profit.director),
            y=director_va_profit[value],
            name=value, 
            marker = dict(color = (potantial_color[ind]),
            line = dict(color = (potantial_color[ind]),width = 3), ) ))
layout = go.Layout(autosize=True,title='Average Vote and Profit for Top 10 Popular Directors',)
fig = go.Figure(data=data,layout=layout)
#plot(fig, show_link=True,filename='director_vote_average_and_profit_3')
py.iplot(fig,filename='director_vote_average_and_profit_3')

The plotly figure is stored in 'plotly_image/director_vote_average_and_profit_3.html' in github.

Those directors are popular for reasons, maybe they do not always make a movie that has a good reputation shown by vote, but they can bring the movie producers profits with great probability. The average profit proportion of these popular directors are all greater than 0, some even reach the profit proportion of 5! From the point of movie producers, they care about relationship between revenue and budget most, which can be shown by the feature value of profit. Thus, making an invitation to popular directors lead to a shorter path for money making.

Production companies of movies also lead to an intuitive thought, since they consist of a whole group of crew. Maybe some of them are not as famous as a hot director, their stable cooperation inside a mutual company will definitely affect the quality of a movie. 

In [166]:
company_count = pd.Series(get_count(merge_an['production_companies']))
company_count = company_count.sort_values(ascending=False) 
company_count=company_count[:10]

We also calculate the average value of several features of 10 production companies that make most movies, since they have a stable number of movie production, and their values avoid a chance of randomness that exist in small companies.

In [170]:
company_overall=get_avg_all(company_count,'production_companies','overall',merge_an)
company_va=get_avg_all(company_count,'production_companies','vote_average',merge_an)
temp_company_pd=pd.merge(company_overall,company_va,on='production_companies')
company_budget=get_avg_all(company_count,'production_companies','budget',merge_an)
temp_company_pd=pd.merge(temp_company_pd,company_budget,on='production_companies')
company_rev=get_avg_all(company_count,'production_companies','revenue',merge_an)
company_pd=pd.merge(temp_company_pd,company_rev,on='production_companies')
company_pd.head(1)

,overall,production_companies,vote_average,budget,revenue
0,4.037447,Warner Bros.,6.583984,5.234810e+07,1.770056e+08


In [184]:
potantial_color=['#b3cdee','#fbb4aa','#fed9b2','#cceba1']
data1=data2 = []
for ind, value in enumerate(['budget','revenue']):
    data1.append(
        go.Bar(
            x=list(company_pd.production_companies),
            y=company_pd[value],
            name=value, 
            marker = dict(color = (potantial_color[ind]),
            line = dict(color = (potantial_color[ind]),width = 3), ) )  )
for ind, value in enumerate(['overall','vote_average']):
    data2.append(
        go.Scatter(
            x=list(company_pd.production_companies),
            y=company_pd[value],
            yaxis='y2',
            name=value,legendgroup=value,
            line = dict(color = (potantial_color[2+ind]),width = 3),), )
layout = go.Layout(autosize=True,    
                   xaxis=dict(title='Companies' ),
                   yaxis=dict(title='Money in Budget or Revenue',showgrid=False, ),
                   yaxis2=dict(title='Average Rating or Vote',overlaying='y',showgrid=False,side='right'),
                   legend=dict(orientation="h", x=.7,y=1.1,),)
data=[data1,data2]
fig = go.Figure(data=data2,layout=layout)
py.iplot(fig, filename='Distribution for Top 10 Movie Companies')

The plotly figure is stored in 'plotly_image/Distribution for Top 10 Movie Companies.html' in github.

Large companies always lead to high revenue and high budget-to-revenue ratio as well. The top movie-maker company Warner Bros. lead to the highest average revenue, and making its role in the market of movie making. Also it seems that rating and vote have the same trend among these companies, since the shape of line plots are similar.

So far we finish the analysis of most intuitive non-value features like genres, directors and production countries. They all help us discover some interesting things, and especially they show the correlation between Amazon rating and TMDB vote, they seem to have a close relationship with each other. Later we will focus on the analysis of value featurs, using mathematical stastical method as well as machine learning algorithms. Also, though the above analysis does not show a causal relationship between review length and rating, intuitively they have a strong relationship between each other, and we will use text propessing method to deeper analyze the inside text features of the reviews.